## Logging on

Use Selenium to visit https://webapps1.chicago.gov/buildingrecords/ and accept the agreement.

> Think about when you use `.find_element...` and when you use `.find_elementSSS...`

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.firefox import GeckoDriverManager

/Users/yikchunlam/.pyenv/versions/3.10.3/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [2]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

/var/folders/86/xmjq1vh10t59mm3v94bs4l_r0000gn/T/ipykernel_92029/3328504527.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())


In [3]:
driver.get('https://webapps1.chicago.gov/buildingrecords/')

In [8]:
driver.find_element(By.ID, 'rbnAgreement1').click()

In [9]:
driver.find_element(By.XPATH, '//*[@id="submit"]').click()

## Searching

Search for **400 E 41ST ST**.

In [11]:
driver.find_element(By.ID, 'fullAddress').send_keys('400 E 41ST ST')

In [12]:
driver.find_element(By.ID, 'submit').click()

## Saving tables with pandas

Use pandas to save a CSV of all **permits** to `Permits - 400 E 41ST ST.csv`. Note that there are **different sections of the page**, not just one long permits table.

> - *Tip: When using `.read_html`, try using `flavor='lxml'` and comparing the results to `flavor='html5lib'`. Which works better?*
> - *Tip: You might need to install `html5lib` using `pip`. If so, you'll need to restart the notebook using **Kernel > Restart** before it will work.*

In [13]:
import pandas as pd

In [21]:
permits = pd.read_html(driver.page_source, flavor='html5lib')

In [30]:
permits[0].to_csv('Permits - 400 E 41ST ST.csv', index= False)

## Saving tables the long way

Save a CSV of all DOB inspections to `Inspections - 400 E 41ST ST.csv`.

This is more complicated than the last one becuse **we also need to save the URL to the inspection** (see how the inspection number is a link?). As a result, you won't be able to use pandas! Instead, you'll need to use a loop and create a list of dictionaries.

You can use Selenium or you can feed the source to BeautifulSoup. You should have approximately 160 rows.

You'll probably need to find the table first, then the rows inside, then the cells inside of each row. You'll probably use lots of list indexing. I might recommend XPath for finding the table.

*Tip: If you get a "list index out of range" error, it's probably due to an issue involving `thead` vs `tbody` elements. What are they? What are they for? What's in them? There are a few ways to troubleshoot it.*

In [66]:
dob = permits[2]
dob_table = driver.find_element(By.ID, 'resultstable_inspections')

inspection_links = []
for item in dob_table.find_elements(By.TAG_NAME, 'a'):
    link = item.get_attribute('href')
    inspection_links.append(link)

dob['links'] = inspection_links

,INSP #,INSPECTION DATE,STATUS,TYPE DESCRIPTION,links
0,13821288,05/16/2022,PASSED,PERMIT INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...
1,13813188,05/16/2022,PASSED,ANNUAL INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...
2,13808208,05/13/2022,FAILED,PERMIT INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...
3,13748085,04/25/2022,FAILED,ANNUAL INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...
4,13684797,03/21/2022,FAILED,ANNUAL INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...
...,...,...,...,...,...
162,25836,07/09/2001,PASSED,SIGN ANNUAL INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...
163,130126,05/09/1997,CLOSED,FIRE PREVENTION PUMPS LEGACY,https://webapps1.chicago.gov/buildingrecords/i...
164,9475223,01/28/1997,CLOSED,ELEVATOR LEGACY INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...
165,9457580,01/21/1997,CLOSED,ELEVATOR LEGACY INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...


In [67]:
dob.to_csv('Inspections - 400 E 41ST ST.csv', index= False)

### Bonus preview of Wednesday's content

**You prrrrrobably shouldn't do this one unless you want a real challenge.**

If you click the inspection number, it'll open up a new window that shows you details of the violations from that visit. Count the number of violations for each visit and save it in a new column called **num_violations**.

Save this file as `Inspections - 400 E 41ST ST - with counts.csv`.

Since clicking the link opens in a new window, we have to say "Hey Selenium, pay attention to that new window!" We do that with `driver.switch_to.window(driver.window_handles[-1])` (each window gets a `window_handle`, and we're just asking the driver to switch to the last one.). A rough sketch of what your code will look like is here:

```python
# Click the link that opens the new window

# Switch to the new window/tab
driver.switch_to.window(driver.window_handles[-1])

# Do your scraping in here

# Close the new window/tab
driver.close()

# Switch back to the original window/tab
driver.switch_to.window(driver.window_handles[0])
```

You'll want to play around with them individually before you try it with the whole set - the ones that pass are very different pages than the ones with violations! There are a few ways to get the number of violations, some easier than others.